In [66]:
import pandas as pd  # Importa la biblioteca pandas.

# Cargar datos de prueba desde un archivo zip.
test_data = pd.read_csv(
    "../files/input/test_data.csv.zip",  # Especifica la ruta relativa al archivo .zip que contiene los datos de prueba.
    index_col=False,  # Indica que no se debe usar ninguna columna como índice del DataFrame.
    compression="zip",  # Especifica que el archivo está comprimido como un .zip.
)

# Cargar datos de entrenamiento desde un archivo zip.
train_data = pd.read_csv(
    "../files/input/train_data.csv.zip",  # Especifica la ruta relativa al archivo .zip que contiene los datos de entrenamiento.
    index_col=False,  # Indica que no se debe usar ninguna columna como índice del DataFrame.
    compression="zip",  # Especifica que el archivo está comprimido como un .zip.
)

In [67]:
#
# limpieza de los datos
#

import numpy as np # importa la biblioteca numpy

# Renombrar la columna 'default payment next month' a 'default'.
test_data.rename(columns={"default payment next month": "default"}, inplace=True)
train_data.rename(columns={"default payment next month": "default"}, inplace=True)

# Eliminar la columna 'ID' de ambos DataFrames.
test_data.drop(columns='ID', inplace=True)
train_data.drop(columns='ID', inplace=True)

# Filtrar y eliminar filas donde los registros con informacion no disponible.
test_data = test_data[(test_data['EDUCATION'] != 0) & (test_data['MARRIAGE'] != 0)]
train_data = train_data[(train_data['EDUCATION'] != 0) & (train_data['MARRIAGE'] != 0)]

# Para la columna EDUCATION, valores > 4 indican niveles superiores de educación, agrupe estos valores en la categoría "others".
test_data['EDUCATION'] = test_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
train_data['EDUCATION'] = train_data['EDUCATION'].apply(lambda x: 4 if x > 4 else x)


In [68]:
#
# separacion de los datos
#

# Elimina la columna 'default' del DataFrame train_data para crear el conjunto de características de entrenamiento.
x_train = train_data.drop(columns="default")

# Extrae la columna 'default' de train_data y la usa como el conjunto de etiquetas de entrenamiento.
y_train = train_data["default"]

# Elimina la columna 'default' del DataFrame test_data para crear el conjunto de características de prueba.
x_test = test_data.drop(columns="default")

# Extrae la columna 'default' de test_data y la usa como el conjunto de etiquetas de prueba.
y_test = test_data["default"]

In [69]:
#
# eleccion modelo y transformaciones
#

# Importaciones de Scikit-Learn para el preprocesador, clasificador, pipeline y métodos de preprocesamiento.
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

# Definición de las columnas categóricas que se transformarán.
categorical_features = ["SEX", "EDUCATION", "MARRIAGE"]

# Configuración del preprocesador utilizando ColumnTransformer.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Aplica OneHotEncoder a las columnas listadas en categorical_features.
        # handle_unknown='ignore': si durante la predicción aparecen valores categóricos nuevos que no estaban presentes en el conjunto de entrenamiento, el transformador los ignorará en lugar de generar un error.
    ],
    remainder="passthrough"  # Especifica que las columnas no listadas en transformers deberían pasarse tal cual al siguiente paso del pipeline.
)

# Creación del pipeline.
pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),  # Primero aplica el preprocesador configurado.
        ('classifier', RandomForestClassifier(random_state=0))  # Luego usa un clasificador de bosque aleatorio para el modelado.
    ]
)

In [70]:
#
# hiperparametros y ajuste
#

from sklearn.model_selection import GridSearchCV  # Importa GridSearchCV para realizar la búsqueda de hiperparámetros.

# Definición de la rejilla de parámetros para el clasificador.
param_grid = {
    'classifier__n_estimators': [100],  # Número de árboles en el bosque.
    'classifier__max_depth': [None],  # Profundidad máxima de los árboles.
    'classifier__min_samples_split': [10],  # Mínimo de muestras para dividir un nodo.
    'classifier__min_samples_leaf': [4],  # Mínimo de muestras en una hoja.
    "classifier__max_features":[23]
}

# Configuración de GridSearchCV.
model = GridSearchCV(
    estimator=pipeline,  # Pipeline con preprocesador y clasificador.
    param_grid=param_grid,  # Conjunto de hiperparámetros a explorar.
    scoring="balanced_accuracy",  # Usa balanced_accuracy para evaluar los modelos.
    cv=10,  # Realiza una validación cruzada con 10 divisiones.
    n_jobs=-1,  # Usa todos los núcleos disponibles para acelerar el proceso.
)

# Ajusta el modelo en el conjunto de entrenamiento.
model.fit(x_train, y_train)

d:\Especializacion Analitica\Predictiva\Laboratorios\2024-2-LAB-01-prediccion-del-default-usando-rf-jfyusty\.venv\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE'])])),
                                       ('classifier',
                                        RandomForestClassifier(random_state=0))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [None],
                         'classifier__max_features': [23],
                         'classifier__min_samples_leaf': [4],
                         'classifier__min_samples_split': [10],
                         'classifier__n_estimators': [100]},
             scoring='balanced_accuracy')

In [71]:
#
# guardar modelo
#

import gzip
import pickle
import os

models_dir = '../files/models'
os.makedirs(models_dir, exist_ok=True)

model_path = "../files/models/model.pkl.gz"

with gzip.open(model_path, "wb") as f:
    pickle.dump(model, f)

model_path

'../files/models/model.pkl.gz'

In [72]:
#
# funcion para guardar  metricas en json
#

import json  # Manejo de archivos JSON.
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score  # Métricas de evaluación.

def calculate_and_save_metrics(model, X_train, X_test, y_train, y_test):
    
    y_train_pred = model.predict(X_train)  # Predice etiquetas para el conjunto de entrenamiento.
    y_test_pred = model.predict(X_test)  # Predice etiquetas para el conjunto de prueba.

    metrics_train = {
        'type': 'metrics',  # Tipo de datos guardados.
        'dataset': 'train',  # Identifica que las métricas corresponden al conjunto de entrenamiento.
        'precision': precision_score(y_train, y_train_pred, zero_division=0),  # Precisión del modelo.
        'balanced_accuracy': balanced_accuracy_score(y_train, y_train_pred),  # Exactitud balanceada.
        'recall': recall_score(y_train, y_train_pred, zero_division=0),  # Sensibilidad o recall.
        'f1_score': f1_score(y_train, y_train_pred, zero_division=0)  # Puntaje F1.
    }

    metrics_test = {
        'type': 'metrics',
        'dataset': 'test',  # Identifica que las métricas corresponden al conjunto de prueba.
        'precision': precision_score(y_test, y_test_pred, zero_division=0),
        'balanced_accuracy': balanced_accuracy_score(y_test, y_test_pred),
        'recall': recall_score(y_test, y_test_pred, zero_division=0),
        'f1_score': f1_score(y_test, y_test_pred, zero_division=0)
    }

    output_dir = os.path.abspath('../files/output')  # Define el directorio donde se guardarán los resultados.
    os.makedirs(output_dir, exist_ok=True)  # Crea el directorio si no existe.

    output_path = os.path.join(output_dir, 'metrics.json')  # Define la ruta completa para el archivo JSON.
    with open(output_path, 'w') as f:  # Abre el archivo en modo escritura.
        f.write(json.dumps(metrics_train) + '\n')  # Escribe las métricas del conjunto de entrenamiento en formato JSON.
        f.write(json.dumps(metrics_test) + '\n')  # Escribe las métricas del conjunto de prueba en formato JSON.

    return metrics_train, metrics_test # Retornar las métricas

In [73]:
#
# funcion para guardar matrices de confucion en json
#

from sklearn.metrics import confusion_matrix  # Importa la función para calcular matrices de confusión.

def calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test):

    # Hacer predicciones
    y_train_pred = model.predict(X_train)  # Predice etiquetas para el conjunto de entrenamiento.
    y_test_pred = model.predict(X_test)  # Predice etiquetas para el conjunto de prueba.

    # Calcular matrices de confusión
    cm_train = confusion_matrix(y_train, y_train_pred)  # Calcula la matriz de confusión para el conjunto de entrenamiento.
    cm_test = confusion_matrix(y_test, y_test_pred)  # Calcula la matriz de confusión para el conjunto de prueba.

    # Convertir las matrices de confusión en formato JSON
    def format_confusion_matrix(cm, dataset_type):
        return {
            'type': 'cm_matrix',  # Tipo de métrica guardada.
            'dataset': dataset_type,  # Indica si es entrenamiento o prueba.
            'true_0': {  # Valores verdaderos para la clase 0.
                'predicted_0': int(cm[0, 0]),  # Predicción correcta para clase 0.
                'predicted_1': int(cm[0, 1])   # Predicción incorrecta para clase 0.
            },
            'true_1': {  # Valores verdaderos para la clase 1.
                'predicted_0': int(cm[1, 0]),  # Predicción incorrecta para clase 1.
                'predicted_1': int(cm[1, 1])   # Predicción correcta para clase 1.
            }
        }

    metrics = [
        format_confusion_matrix(cm_train, 'train'),  # Matriz de confusión para entrenamiento.
        format_confusion_matrix(cm_test, 'test')    # Matriz de confusión para prueba.
    ]

    # Guardar las matrices de confusión en el archivo JSON
    output_path = '../files/output/metrics.json'  # Ruta del archivo JSON.
    with open(output_path, 'a') as f:  # Abre el archivo en modo agregar ('a').
        for metric in metrics:
            f.write(json.dumps(metric) + '\n')  # Convierte el diccionario en JSON y lo guarda.

    return cm_train, cm_test # Devuelve las matrices de confusión originales.

In [74]:
#
# funcion ejecutura
#

def main(model, X_train, X_test, y_train, y_test):

    # Crear el directorio de salida si no existe
    import os
    os.makedirs('../files/output', exist_ok=True)

    # Calcular y guardar las métricas
    metrics_train, metrics_test = calculate_and_save_metrics(model, X_train, X_test, y_train, y_test)

    # Imprimir las métricas
    print("Métricas del conjunto de entrenamiento:")
    for key, value in metrics_train.items():
        print(f"{key}: {value}")

    print("\nMétricas del conjunto de prueba:")
    for key, value in metrics_test.items():
        print(f"{key}: {value}")

    # Calcular, guardar e imprimir las matrices de confusión
    cm_train, cm_test = calculate_and_save_confusion_matrices(model, X_train, X_test, y_train, y_test)

    print("\nMatriz de confusión del conjunto de entrenamiento:")
    print(cm_train)

    print("\nMatriz de confusión del conjunto de prueba:")
    print(cm_test)



In [75]:
# Ejemplo de uso:
main(model, x_train, x_test, y_train, y_test)

Métricas del conjunto de entrenamiento:
type: metrics
dataset: train
precision: 0.953005115089514
balanced_accuracy: 0.8109205266747785
recall: 0.6308994708994708
f1_score: 0.7592003056156883

Métricas del conjunto de prueba:
type: metrics
dataset: test
precision: 0.6635354397950469
balanced_accuracy: 0.6759776140560241
recall: 0.40766002098635884
f1_score: 0.5050373740656483

Matriz de confusión del conjunto de entrenamiento:
[[16081   147]
 [ 1744  2981]]

Matriz de confusión del conjunto de prueba:
[[6679  394]
 [1129  777]]
